In [1]:
import numpy as np
from skimage import io
from skimage.feature import peak_local_max
import pandas as pd
import os


In [ ]:
#@title Feature Extraction Definitions
def extract_spot_features(path):

  # ------------------------------
  # Image processing
  # ------------------------------

  ### import

  data = io.imread(path)
  [n_images, length, width] = np.shape(data)

  # ------------------------------
  # Feature extraction
  # ------------------------------

  ### timestamp for non-temporal features
  t = 90

  # Area, count, and fluorescence over time
 
  maxima = [peak_local_max(data[n, :,:], min_distance=2, threshold_rel= .01, threshold_abs=150) for n in range(n_images)]
  n_spots_over_time = [len(m) for m in maxima]
  thresh = 150
  area_over_time = [np.sum(data[t] > thresh) / (length * width) for t in range(n_images)]
  bulk_fluorescence_over_time = [np.mean(data[t]) for t in range(n_images)]

  ### timestamp for non-temporal features
  t = 90

  ### spot count
  n_spots = n_spots_over_time[t]
  max_n_spots = max(n_spots_over_time)

  ### bulk fluorescence
  bulk_fluorescence = bulk_fluorescence_over_time[t]

  ### percent area
  percent_area = area_over_time[t] if n_spots > 0 else 0

  ### spot size (total pixel area w/ signal above threshold)
  avg_spot_size = percent_area / n_spots if n_spots > 0 else 0

  ### time to threshold
  try:
    # time_to_threshold = [np.max(data[t,:,:]) >= 5 for t in range(n_images)].index(True)
    time_to_threshold = np.argmax(n_spots_over_time)/3
  except ValueError:
    time_to_threshold = n_images

  ### max rate of change of spot count over 10 seconds
  spot_counts_trimmed = [x for x in n_spots_over_time if x != 0]
  spot_count_deltas = [spot_counts_trimmed[i + 10] - spot_counts_trimmed[i] for i in range(len(spot_counts_trimmed) - 10)]
  max_spot_change = max(spot_count_deltas) if len(spot_count_deltas) > 0 else 0

  ### max rate of change of spot area over 10 seconds
  area_deltas = [area_over_time[i + 10] - area_over_time[i] for i in range(len(area_over_time) - 10)]
  max_area_change = max(area_deltas) if len(area_deltas) > 0 else 0

  ### max rate of change of bulk fluorescence over 10 seconds
  fluorescence_deltas = [bulk_fluorescence_over_time[i + 10] - bulk_fluorescence_over_time[i] for i in range(len(bulk_fluorescence_over_time) - 10)]
  max_fluorescence_change = max(fluorescence_deltas) if len(fluorescence_deltas) > 0 else 0

  # construct features map

  features = {
      "n_spots": n_spots,
      "max_n_spots": max_n_spots,
      "bulk_fluorescence": bulk_fluorescence,
      "avg_spot_size": avg_spot_size,
      "percent_area": percent_area,
      "time_to_threshold": time_to_threshold,
      "max_spot_change": max_spot_change,
      "max_area_change": max_area_change,
      "max_fluorescence_change": max_fluorescence_change
  }

  # for i in range(240, n_images, 60):
  #   features[f"n_spots_{i}"] = n_spots_over_time[i]
  #   features[f"bulk_fluorescence_{i}"] = bulk_fluorescence_over_time[i]
  #   features[f"percent_area_{i}"] = area_over_time[i] if n_spots > 0 else 0

  return features

In [9]:
#@title Experiments
dir = r"../RPA on glass slides/100_serial/processed"


experiments = {
    "0": [
        '0-1.tif',
        '0.tif'
    ],
    "1000": [
        '1000.tif',
        '1000-1.tif',
        '1000-2.tif'
    ],
    "10000": [
        '10000-2.tif',
        '10000-1.tif',
        '10000.tif'

    ],
    "100000": [
        '100000-1.tif',
        '100000.tif'
    ],
    "20": [
        '20.tif'
    ],
    "200": [
        '200-1.tif',
        '200-2.tif',
        '200.tif'

    ],
    "2000": [
        '2000-1.tif',
        '2000.tif'

    ],
    "3000": [
        '3000.tif'

    ],
    "30000": [
        '30000-1.tif',
        '30000-2.tif',
        '30000-3.tif',
        '30000-4.tif',
        '30000.tif'

    ],
    "5000": [
        '5000-1.tif',
        '5000-2.tif',
        '5000-3.tif',
        '5000-4.tif',
        '5000-5.tif',
        '5000.tif'

    ],

    "40": [
        '40.tif'

    ],
    "500": [
        '500-1.tif',
        '500.tif'

    ],
    "50000": [
        '50000-1.tif',
        '50000.tif'

    ],

    "80": [
        '80.tif'

    ],
    "8000": [
        '8000-1.tif',
        '8000.tif'

    ],
    "80000": [
        '80000-1.tif',
        '80000.tif'

    ]

}


In [10]:
#@title Extract Features to CSV

features_csv = "structured_data.csv"

if not os.path.exists(features_csv):
  features = {}

  for n_copies in experiments:
    features[n_copies] = []
    for exp in experiments[n_copies]:
      image_stack = os.path.join(dir, n_copies, exp)
      features[n_copies].append(extract_spot_features(image_stack))

  # Parse the data into a DataFrame
  data = []
  for copy_number, records in features.items():
      for record in records:
          record['copy_number'] = copy_number
          data.append(record)

  df = pd.DataFrame(data)

  # Convert copy numbers to log scale
  df['log_copy_number'] = df['copy_number'].replace({'cps': '', ',': ''}, regex=True).astype(float)
  df['log_copy_number'] = np.log10(df['log_copy_number'])

  df.to_csv(features_csv, index=False)
else:
  df = pd.read_csv(features_csv)

df.replace(-np.inf, np.nan, inplace=True)
df.dropna(inplace=True)

df.head()

c:\Users\CDM\miniconda3\envs\ANSA_new\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


,n_spots,max_n_spots,bulk_fluorescence,avg_spot_size,percent_area,time_to_threshold,max_spot_change,max_area_change,max_fluorescence_change,copy_number,log_copy_number
2,537,576,56.066792,0.000283,0.152094,34.000000,264,0.062094,22.959905,1000,3.0
3,281,293,74.177323,0.000471,0.132277,41.666667,123,0.042786,25.343478,1000,3.0
4,236,260,35.715783,0.000326,0.076926,41.666667,106,0.030125,14.762014,1000,3.0
5,476,482,133.732320,0.000482,0.229381,31.000000,239,0.079694,36.214404,10000,4.0
6,513,531,128.079893,0.000467,0.239780,27.666667,289,0.085354,36.735576,10000,4.0
